In [1]:
import cantera as ct
import os 
import re
from rmgpy.molecule import Molecule
from rmgpy.reaction import Reaction
from rmgpy.species import Species
from rmgpy.data.kinetics.family import KineticsFamily
from rmgpy.data.rmg import RMGDatabase
from rmgpy import settings
from rmgpy.data.base import Database
from rmgpy.chemkin import load_species_dictionary

Make a species dictionary

In [2]:
unneeded_species = ['CHF(T)',
'CF2(T)',
'CH3CF(T)',
'CH2FCH(T)',
'CH2FCF(T)',
'CHF2CH(S)',
'CHF2CF(T)',
'CF3CH(S)',
'CF3CF(T)',
'CF3CO']


species_dictionary = {}


file = './hcof_anl0_with_c2_20231126.yaml'

with open(file, 'r') as f: 
    lines = f.readlines()
#species are in lines 12-103
for line in lines[11:103]:
    split_species = line.split(', #')
    [label, smiles] = split_species
    
    #handle the label first
    label = label.replace(' ', '')
        
    #now the smiles
    smiles = smiles.replace(' ', '').replace('\n', '').replace(',','')
    match=re.search('([a-z]+)', smiles)
    if match:
        also_remove = match.group(1)
        smiles = smiles.replace(also_remove, '')
        
    #now save to species dictionary
    if label in unneeded_species: 
        continue
    else:
        species_dictionary[label] = smiles
        

In [3]:
#add the last couple of stragglers

species_dictionary['H']= "[H]"
species_dictionary['H2']= "[H][H]"
species_dictionary['CH']= "[CH]"
species_dictionary['CH2']= "[CH2]"
species_dictionary['CH3']= "[CH3]"
species_dictionary['CH4']= "C"
species_dictionary['O']= "[O]"
species_dictionary['OH']= "[OH]"
species_dictionary['H2O']= "O"
species_dictionary['O2']= "[O][O]"
species_dictionary['HO2']= "[O]O"
species_dictionary['H2O2']= "OO"
species_dictionary['CO']= "[C-]#[O+]"
species_dictionary['CO2']= "O=C=O"
species_dictionary['HCO']= "[CH]=O"
species_dictionary['CH2O'] = "C=O"

Now let's make the reaction.py

But we need the degeneracy first

In [4]:
gas = ct.Solution('/work/westgroup/nora/Code/projects/PFAS/ESSCI/models/ANL_Brown5/format_reactions/H_abstraction_/hcof_anl0_with_c2_20231126.yaml')

In [5]:
#load in the database
database = RMGDatabase()
database.load(
            path = settings['database.directory'],
            thermo_libraries = ['Klippenstein_Glarborg2016', 'BurkeH2O2', 'thermo_DFT_CCSDTF12_BAC', 
                               'DFT_QCI_thermo',
                           'primaryThermoLibrary', 'primaryNS', 'NitrogenCurran', 'NOx2018', 'FFCM1(-)',
'SulfurLibrary', 'SulfurGlarborgH2S','SABIC_aromatics'],
            transport_libraries = [],
            reaction_libraries = [],
            seed_mechanisms = [],#['BurkeH2O2inN2','ERC-FoundationFuelv0.9'],
            kinetics_families = 'all',
            kinetics_depositories = ['training'],
            #frequenciesLibraries = self.statmechLibraries,
            depository = False, # Don't bother loading the depository information, as we don't use it
        )

In [ ]:
#H-abstraction 0-40(including)
#F-abstraction 41-80
#Substitution 81, 82, 83, 84
#will have to make a Brown halogens pdep library too

In [16]:
rmg_family = 'H_Abstraction'
H_abstraction_lines = []
continued_count = 197 #3784 was last index in H_abstraction reactions.py #116 is last one is Brown_dif_seed
for index, rxn in enumerate(gas.reactions()[81:]):
    continued_count+=1
 
    #get rate parameters
    A = "{:e}".format(rxn.rate.pre_exponential_factor*(100*100*100)/1000) #convert from m3 / kmol s to cm3 / mol s
    n = rxn.rate.temperature_exponent
    Ea = rxn.rate.activation_energy/(1000*1000) #convert from J/kmol to kJ/mol
    
#     #get the degeneracy using RMG
#     reactants = list(rxn.reactants.keys())
#     products = list(rxn.products.keys())

#     reactant_Molecule = [Molecule(smiles=species_dictionary[x]) for x in reactants]
#     product_Molecule = [Molecule(smiles=species_dictionary[x]) for x in products]

#     template_reaction = database.kinetics.families[rmg_family].generate_reactions(reactants=reactant_Molecule, products=product_Molecule)[0] #just choose the first
#     degeneracy = database.kinetics.families[rmg_family].calculate_degeneracy(template_reaction)
    
    
#     #now match to species already existing in reactions.py
#     labeled_adjacencies = get_labeled_adj_lists(rxn)
#     switch_labels = get_matching_species_label(labeled_adjacencies)

    
#     #and you should change the labels to existing labels in the reactions.py file if the species already exists there 
#     final_reactants = []
#     for reactant in reactants: 
#         final_label = switch_labels[reactant]
#         final_reactants.append(final_label)

#     equation_string = ''
#     for reactant in final_reactants: 
#         equation_string+=f'{reactant} + '
#     equation_string=equation_string[:-3] + ' <=> ' #remove the last bit when the loop finishes and add a '<=>'


#     final_products = []
#     for product in products: 
#         final_label = switch_labels[product]
#         final_products.append(final_label)

#     #now add to the equation string
#     for product in final_products: 
#         equation_string+=f'{product} + '
#     equation_string=equation_string[:-3] #remove the last bit when the loop finishes and add a '<=>'


    equation_string = str(rxn)


    f_string = f'''
entry(
    index = {continued_count},
    label = "{equation_string}",
    degeneracy = 1.0,
    kinetics = Arrhenius(A=({A},'cm^3/(mol*s)'), n={n}, Ea=({Ea},'kJ/mol'), T0=(1,'K'), Tmin=(300,'K'), Tmax=(2000,'K')),
    rank = 5,
    shortDesc = """ANL0 method from Brown""",
    longDesc = 
"""
Electronic structures done using ANL0 compound method. 
Torsional scans with  M06-2X/cc-pVTZ. 
""",
)
    '''
    print(f_string)


entry(
    index = 198,
    label = "CH4 + F <=> CH3F + H",
    degeneracy = 1.0,
    kinetics = Arrhenius(A=(4.570000e+04,'cm^3/(mol*s)'), n=2.58, Ea=(84.9,'kJ/mol'), T0=(1,'K'), Tmin=(300,'K'), Tmax=(2000,'K')),
    rank = 5,
    shortDesc = """ANL0 method from Brown""",
    longDesc = 
"""
Electronic structures done using ANL0 compound method. 
Torsional scans with  M06-2X/cc-pVTZ. 
""",
)
    

entry(
    index = 199,
    label = "CH3F + F <=> CH2F2 + H",
    degeneracy = 1.0,
    kinetics = Arrhenius(A=(1.470000e+04,'cm^3/(mol*s)'), n=2.61, Ea=(87.4,'kJ/mol'), T0=(1,'K'), Tmin=(300,'K'), Tmax=(2000,'K')),
    rank = 5,
    shortDesc = """ANL0 method from Brown""",
    longDesc = 
"""
Electronic structures done using ANL0 compound method. 
Torsional scans with  M06-2X/cc-pVTZ. 
""",
)
    

entry(
    index = 200,
    label = "CH2F2 + F <=> CHF3 + H",
    degeneracy = 1.0,
    kinetics = Arrhenius(A=(1.620000e+03,'cm^3/(mol*s)'), n=2.73, Ea=(116.8,'kJ/mol'), T0=(1,'K'), Tmin=(300,

AttributeError: 'cantera._cantera.PlogRate' object has no attribute 'pre_exponential_factor'

In [ ]:
rmg_family = 'H_Abstraction'
H_abstraction_lines = []
continued_count = 116 #3784 was last index in H_abstraction reactions.py #116 is last one is Brown_dif_seed
for index, rxn in enumerate(gas.reactions()[0:41]):
    continued_count+=1
 
    #get rate parameters
    A = "{:e}".format(rxn.rate.pre_exponential_factor*(100*100*100)/1000) #convert from m3 / kmol s to cm3 / mol s
    n = rxn.rate.temperature_exponent
    Ea = rxn.rate.activation_energy/(1000*1000) #convert from J/kmol to kJ/mol
    
    #get the degeneracy using RMG
    reactants = list(rxn.reactants.keys())
    products = list(rxn.products.keys())

    reactant_Molecule = [Molecule(smiles=species_dictionary[x]) for x in reactants]
    product_Molecule = [Molecule(smiles=species_dictionary[x]) for x in products]

    template_reaction = database.kinetics.families[rmg_family].generate_reactions(reactants=reactant_Molecule, products=product_Molecule)[0] #just choose the first
    degeneracy = database.kinetics.families[rmg_family].calculate_degeneracy(template_reaction)
    
    
    #now match to species already existing in reactions.py
    labeled_adjacencies = get_labeled_adj_lists(rxn)
    switch_labels = get_matching_species_label(labeled_adjacencies)

    
    #and you should change the labels to existing labels in the reactions.py file if the species already exists there 
    final_reactants = []
    for reactant in reactants: 
        final_label = switch_labels[reactant]
        final_reactants.append(final_label)

    equation_string = ''
    for reactant in final_reactants: 
        equation_string+=f'{reactant} + '
    equation_string=equation_string[:-3] + ' <=> ' #remove the last bit when the loop finishes and add a '<=>'


    final_products = []
    for product in products: 
        final_label = switch_labels[product]
        final_products.append(final_label)

    #now add to the equation string
    for product in final_products: 
        equation_string+=f'{product} + '
    equation_string=equation_string[:-3] #remove the last bit when the loop finishes and add a '<=>'

    f_string = f'''
entry(
    index = {continued_count},
    label = "{equation_string}",
    degeneracy = {degeneracy},
    kinetics = Arrhenius(A=({A},'cm^3/(mol*s)'), n={n}, Ea=({Ea},'kJ/mol'), T0=(1,'K'), Tmin=(300,'K'), Tmax=(2000,'K')),
    rank = 5,
    shortDesc = """ANL0 method from Brown""",
    longDesc = 
"""
Electronic structures done using ANL0 compound method. 
Torsional scans with  M06-2X/cc-pVTZ. 
""",
)
    '''
    print(f_string)

In [6]:
#existing H abstraction training dictionary 
e_path = '/home/khalil.nor/Code/RMG-database/input/kinetics/families/H_Abstraction/training/dictionary.txt'
existing_dict = load_species_dictionary(e_path)

brown_path = './species_dictionary_edited.txt'
brown_dict = load_species_dictionary(brown_path)

In [7]:

def get_labeled_adj_lists(rxn): 
    """
    given a reaction, use RMG's .get_labeled_reactants_and_products to get the actual adjacency lists with all the *#s 

    """
    
    labeled_adjacencies = {}

    reactants = list(rxn.reactants.keys())
    products = list(rxn.products.keys())

    #make all Molecule objects
    reactant_Molecule = [Molecule(smiles=species_dictionary[x]) for x in reactants]
    product_Molecule = [Molecule(smiles=species_dictionary[x]) for x in products]

    reactant_Species = [Species(molecule=[Molecule(smiles=species_dictionary[x])], label=x) for x in reactants]
    product_Species = [Species(molecule=[Molecule(smiles=species_dictionary[x])], label=x) for x in products]


    generated_labels = database.kinetics.families[rmg_family].get_labeled_reactants_and_products(reactant_Molecule,product_Molecule, relabel_atoms=True)

    #reactants
    for reactant_s in reactant_Species: 
        for reactant in generated_labels[0]: #generated_labels[0] is list of reactants from .get_labeled_reactants_and_products
            if reactant.is_isomorphic(reactant_s.molecule[0]):  #let's match which generated label goes with which reactant
                reactant_label = reactant_s.label
                if reactant_label not in labeled_adjacencies.keys():
                    labeled_adjacencies[reactant_label]=[reactant.to_adjacency_list(), reactant_s]

    #products 
    for product_s in product_Species: 
        for product in generated_labels[1]: 
            if product.is_isomorphic(product_s.molecule[0]):
                product_label = product_s.label
                if product_label not in labeled_adjacencies.keys():
                    labeled_adjacencies[product_label]=[product.to_adjacency_list(), product_s]
    
    return labeled_adjacencies #labeled_adjacencies is a dictionary with key = species label, value = [generated adj list, Species object]

def get_matching_species_label(labeled_adjacencies):   

    switch_labels = {}
    for key, value in labeled_adjacencies.items():
        [adj_list, spec] = value
        starred = re.findall('(\*[0-9] [A-Z])', adj_list)
        for label, species in existing_dict.items(): #let's see if theres an existing species name that precisely matches this adj list
            if species.is_isomorphic(spec):
                flag = 0 
                for starred_str in starred:
                    test_str = species.to_adjacency_list()
                    if starred_str in test_str: #that there is an existing species in dictionary.txt that has a "*# Atom_symbol"
                        pass
                    else: 
                        flag+=1
                        
                    #make sure theres no extra stars in the existing species adj list that we're looking at 
                    already_present = re.findall('(\*[0-9] [A-Z])', test_str)
                    if len(already_present)>len(starred):
                        #this means theres an extra
                        flag+=1
                if flag == 0:
                    switch_labels[key]=label #if everything has worked out...
                    # print(species.to_adjacency_list())
                    # print('matched to')
                    # print(adj_list)

    return switch_labels

In [84]:
gas.reactions()[0]

CH3F + H <=> CH2F + H2    <Reaction(Arrhenius)>

In [13]:
#o = get_labeled_adj_lists(gas.reactions()[0])

for k,v in species_dictionary.items():
    print(k)
    print(Molecule(smiles=v).to_adjacency_list())

F
multiplicity 2
1 F u1 p3 c0

F2
1 F u0 p3 c0 {2,S}
2 F u0 p3 c0 {1,S}

HF
1 F u0 p3 c0 {2,S}
2 H u0 p0 c0 {1,S}

CF
multiplicity 2
1 C u1 p1 c0 {2,S}
2 F u0 p3 c0 {1,S}

CHF
multiplicity 3
1 C u2 p0 c0 {2,S} {3,S}
2 F u0 p3 c0 {1,S}
3 H u0 p0 c0 {1,S}

CH2F
multiplicity 2
1 C u1 p0 c0 {2,S} {3,S} {4,S}
2 F u0 p3 c0 {1,S}
3 H u0 p0 c0 {1,S}
4 H u0 p0 c0 {1,S}

CH3F
1 C u0 p0 c0 {2,S} {3,S} {4,S} {5,S}
2 F u0 p3 c0 {1,S}
3 H u0 p0 c0 {1,S}
4 H u0 p0 c0 {1,S}
5 H u0 p0 c0 {1,S}

CF2
multiplicity 3
1 F u0 p3 c0 {2,S}
2 C u2 p0 c0 {1,S} {3,S}
3 F u0 p3 c0 {2,S}

CHF2
multiplicity 2
1 F u0 p3 c0 {2,S}
2 C u1 p0 c0 {1,S} {3,S} {4,S}
3 F u0 p3 c0 {2,S}
4 H u0 p0 c0 {2,S}

CH2F2
1 F u0 p3 c0 {2,S}
2 C u0 p0 c0 {1,S} {3,S} {4,S} {5,S}
3 F u0 p3 c0 {2,S}
4 H u0 p0 c0 {2,S}
5 H u0 p0 c0 {2,S}

CF3
multiplicity 2
1 F u0 p3 c0 {2,S}
2 C u1 p0 c0 {1,S} {3,S} {4,S}
3 F u0 p3 c0 {2,S}
4 F u0 p3 c0 {2,S}

CHF3
1 F u0 p3 c0 {2,S}
2 C u0 p0 c0 {1,S} {3,S} {4,S} {5,S}
3 F u0 p3 c0 {2,S}
4 F u0 p3 c0 {2,S

In [101]:
gas.reactions()[0]

CH3F + H <=> CH2F + H2    <Reaction(Arrhenius)>

In [132]:
rxn = gas.reactions()[0]

reactants = list(rxn.reactants.keys())
products = list(rxn.products.keys())

#make all Molecule objects
reactant_Molecule = [Molecule(smiles=species_dictionary[x]) for x in reactants]
product_Molecule = [Molecule(smiles=species_dictionary[x]) for x in products]

reactant_Species = [Species(label=x, molecule=[Molecule(smiles=species_dictionary[x])]) for x in reactants]
product_Species = [Species(label=x, molecule=[Molecule(smiles=species_dictionary[x])]) for x in products]



In [118]:
reactant_Species

[Species(label="CH3F", molecule=[Molecule(smiles="CF")], molecular_weight=(34.033,'amu')),
 Species(label="H", molecule=[Molecule(smiles="[H]")], molecular_weight=(1.00797,'amu'))]

In [120]:
tr = database.kinetics.generate_reactions_from_families(reactant_Species, products=product_Species, only_families=[rmg_family])

[TemplateReaction(reactants=[Species(label="", molecule=[Molecule(smiles="CF")], molecular_weight=(34.033,'amu')), Species(label="", molecule=[Molecule(smiles="[H]")], molecular_weight=(1.00797,'amu'))], products=[Species(label="", molecule=[Molecule(smiles="[H][H]")], molecular_weight=(2.01594,'amu')), Species(label="", molecule=[Molecule(smiles="[CH2]F")], molecular_weight=(33.025,'amu'))], degeneracy=3.0, pairs=[[Species(label="", molecule=[Molecule(smiles="CF")], molecular_weight=(34.033,'amu')), Species(label="", molecule=[Molecule(smiles="[CH2]F")], molecular_weight=(33.025,'amu'))], [Species(label="", molecule=[Molecule(smiles="[H]")], molecular_weight=(1.00797,'amu')), Species(label="", molecule=[Molecule(smiles="[H][H]")], molecular_weight=(2.01594,'amu'))]], family='H_Abstraction', template=['Root_N-3R->F_N-3BrCClHINOS->C_Ext-1R-R_Sp-4R!H-1R_N-3BrClHINOS->Cl_N-1R->S_N-4R!H->Cl_N-3BrHINOS->S_N-3BrHINO->I_N-4BrCFNOS->O_4BrCFNS-u0_N-4BrCFNS->C_N-3BrHNO->Br_3HNO-u1_N-3HNO->O_4BrF

In [127]:

#template_rxns = database.kinetics.families[rmg_family].get_reaction_template(

database.kinetics.families[rmg_family].get_reaction_template_labels(tr[0])

UndeterminableKineticsError: (TemplateReaction(reactants=[Species(label="CF", molecule=[Molecule(smiles="CF")], molecular_weight=(34.033,'amu')), Species(label="[H]", molecule=[Molecule(smiles="[H]")], molecular_weight=(1.00797,'amu'))], products=[Species(label="[H][H]", molecule=[Molecule(smiles="[H][H]")], molecular_weight=(2.01594,'amu')), Species(label="[CH2]F", molecule=[Molecule(smiles="[CH2]F")], molecular_weight=(33.025,'amu'))], degeneracy=3.0, pairs=[[Species(label="CF", molecule=[Molecule(smiles="CF")], molecular_weight=(34.033,'amu')), Species(label="[CH2]F", molecule=[Molecule(smiles="[CH2]F")], molecular_weight=(33.025,'amu'))], [Species(label="[H]", molecule=[Molecule(smiles="[H]")], molecular_weight=(1.00797,'amu')), Species(label="[H][H]", molecule=[Molecule(smiles="[H][H]")], molecular_weight=(2.01594,'amu'))]], family='H_Abstraction', template=['Root_N-3R->F_N-3BrCClHINOS->C_Ext-1R-R_Sp-4R!H-1R_N-3BrClHINOS->Cl_N-1R->S_N-4R!H->Cl_N-3BrHINOS->S_N-3BrHINO->I_N-4BrCFNOS

In [133]:
generated_labels = database.kinetics.families[rmg_family].get_labeled_reactants_and_products(reactant_Molecule,product_Molecule, relabel_atoms=True)

In [147]:
print(generated_labels[1][].to_adjacency_list())

1    F u0 p3 c0 {2,S}
2 *1 C u0 p0 c0 {1,S} {3,S} {4,S} {5,S}
3 *2 H u0 p0 c0 {2,S}
4    H u0 p0 c0 {2,S}
5    H u0 p0 c0 {2,S}



In [131]:
reactant_Molecule

[Species(label="", molecule=[Molecule(smiles="CF")], molecular_weight=(34.033,'amu')),
 Species(label="", molecule=[Molecule(smiles="[H]")], molecular_weight=(1.00797,'amu'))]

In [16]:
#template_reaction = database.kinetics.families[rmg_family].generate_reactions(reactants=reactant_Molecule, products=product_Molecule)[0] #just choose the first
x = database.kinetics.families[rmg_family].get_training_set()


In [17]:
for y in x:
    print(y)

H2O2 + C4H9O <=> HO2 + C4H10O
H2O2 + C4H9O-2 <=> HO2 + C4H10O-2
H2O2 + C4H9O-3 <=> HO2 + C4H10O-3
H2O2 + C4H9O-4 <=> HO2 + C4H10O-4
H2O2 + C4H9O-5 <=> HO2 + C4H10O-5
H2O2 + C4H9O-6 <=> HO2 + C4H10O-6
H2O2 + C4H9O-7 <=> HO2 + C4H10O-7
H2O2 + C4H9O-8 <=> HO2 + C4H10O-8
H2O2 + C4H9O-9 <=> HO2 + C4H10O-9
CH2O + C4H7 <=> CHO + C4H8
C3H8 + C4H9O-10 <=> C3H7 + C4H10O-10
C4H10O-11 + C3H7-2 <=> C4H9O-11 + C3H8-2
C4H8-2 + C4H9O-12 <=> C4H7-2 + C4H10O-12
C4H8-2 + C4H9O-10 <=> C4H7-2 + C4H10O-10
C4H8-2 + C4H9O-13 <=> C4H7-2 + C4H10O-13
C4H8-2 + C4H9O-14 <=> C4H7-2 + C4H10O-14
C3H6 + C4H9O-12 <=> C3H5 + C4H10O-12
C3H6 + C4H9O-10 <=> C3H5 + C4H10O-10
C3H6 + C4H9O-13 <=> C3H5 + C4H10O-13
C3H6 + C4H9O-14 <=> C3H5 + C4H10O-14
C2H6 + C4H9O-12 <=> C2H5 + C4H10O-12
C4H10O-15 + C2H5-2 <=> C4H9O-15 + C2H6-2
C4H10O-11 + C2H5-2 <=> C4H9O-11 + C2H6-2
C2H6 + C4H9O-14 <=> C2H5 + C4H10O-14
C4H10O-15 + C2H3 <=> C4H9O-15 + C2H4
C4H10O-16 + C3H5-2 <=> C4H9O-16 + C3H6-2
C4H10O-11 + C3H5-2 <=> C4H9O-11 + C3H6-2
C3H6O 

In [19]:
new_reactions = []
for ind, y in enumerate(x):
     if ind > 3784:
            new_reactions.append(y)

In [20]:
old_reactions = []
for ind, y in enumerate(x):
     if ind < 3784:
            old_reactions.append(y)

In [22]:
keep = []
toss = []
for old_rxn in old_reactions:
    flag=0
    for new_rxn in new_reactions:
        if old_rxn.is_isomorphic(new_rxn):
            print(old_rxn, new_rxn)
            toss.append(old_rxn)
            flag+=1            
    if flag==0:
        keep.append(old_rxn)
        

CH3F + O <=> CH2F + HO-2 CH3F + O <=> CH2F-2 + HO-4
CH2F2 + O <=> CHF2 + HO-2 CH2F2 + O <=> CHF2-2 + HO-4
CH4 + F <=> CH3 + FH CH4 + F <=> CH3-2 + HF
CH3F + F <=> CH2F + FH CH3F + F <=> CH2F-2 + HF
CH2F2 + F <=> CHF2 + FH CH2F2 + F <=> CHF2-2 + HF
CH2O + F <=> CHO + FH CH2O + F <=> CHO-2 + HF
O + CH3F <=> HO-2 + CH2F CH3F + O <=> CH2F-2 + HO-4
O + CH2F2 <=> HO-2 + CHF2 CH2F2 + O <=> CHF2-2 + HO-4
F + CH2F2 <=> FH + CHF2 CH2F2 + F <=> CHF2-2 + HF
CF3-2 + CH2F2 <=> CHF3-2 + CHF2 CF3-2 + CH2F2 <=> CHF2-2 + CHF3
CHF2-2 + CH2O <=> CH2F2-2 + CHO CH2O + CHF2-2 <=> CH2F2 + CHO-2
HO + CH3F <=> H2O + CH2F CH3F + HO <=> CH2F-2 + H2O-2
HO + CHFO <=> H2O + CFO CHFO + HO <=> CFO-2 + H2O-2
CF3-2 + H2O2 <=> CHF3-2 + HO2 CF3-2 + H2O2 <=> CHF3 + HO2-2
CH2F-2 + H2O2 <=> CH3F-2 + HO2 CH2F-2 + H2O2 <=> CH3F + HO2-2
H + CH2F2 <=> H2 + CHF2 CH2F2 + H <=> CHF2-2 + H2-2
F + CHF3 <=> FH + CF3 CHF3 + F <=> CF3-2 + HF
CF3-2 + HO-4 <=> CHF3-2 + O-2 CF3-2 + HO-4 <=> CHF3 + O
CFO-2 + CH3F <=> CHFO-2 + CH2F CH3F + CF

In [26]:
for x in toss:
    print(x)

CH3F + O <=> CH2F + HO-2
CH2F2 + O <=> CHF2 + HO-2
CH4 + F <=> CH3 + FH
CH3F + F <=> CH2F + FH
CH2F2 + F <=> CHF2 + FH
CH2O + F <=> CHO + FH
O + CH3F <=> HO-2 + CH2F
O + CH2F2 <=> HO-2 + CHF2
F + CH2F2 <=> FH + CHF2
CF3-2 + CH2F2 <=> CHF3-2 + CHF2
CHF2-2 + CH2O <=> CH2F2-2 + CHO
HO + CH3F <=> H2O + CH2F
HO + CHFO <=> H2O + CFO
CF3-2 + H2O2 <=> CHF3-2 + HO2
CH2F-2 + H2O2 <=> CH3F-2 + HO2
H + CH2F2 <=> H2 + CHF2
F + CHF3 <=> FH + CF3
CF3-2 + HO-4 <=> CHF3-2 + O-2
CFO-2 + CH3F <=> CHFO-2 + CH2F
CF3-2 + CH4 <=> CHF3-2 + CH3
CHF2-2 + H2O2 <=> CH2F2-2 + HO2
CH2F-2 + CH2O <=> CH3F-2 + CHO
CHF2-2 + CH3F <=> CH2F2-2 + CH2F
CF3-2 + CH2O <=> CHF3-2 + CHO
HO + CH2F2 <=> H2O + CHF2
CF3-2 + CHFO <=> CHF3-2 + CFO
H + CH3F <=> H2 + CH2F
CFO-2 + H2O2 <=> CHFO-2 + HO2
CH3-2 + CH2F2 <=> CH4-2 + CHF2
CF3-2 + CH3F <=> CHF3-2 + CH2F
CH3-2 + CHFO <=> CH4-2 + CFO
CH3-2 + CH3F <=> CH4-2 + CH2F
CFO-2 + CH2F2 <=> CHFO-2 + CHF2
F + CHFO <=> FH + CFO
HO + CHF3 <=> H2O + CF3
CF3-2 + H2-2 <=> CHF3-2 + H-2
O + CHFO <

In [ ]:
# tossing 

#CH3F + O <=> CH2F + HO-2
# CH2F2 + O <=> CHF2 + HO-2
# CH4 + F <=> CH3 + FH
# CH3F + F <=> CH2F + FH
# CH2F2 + F <=> CHF2 + FH
# CH2O + F <=> CHO + FH
# O + CH3F <=> HO-2 + CH2F
# O + CH2F2 <=> HO-2 + CHF2
# F + CH2F2 <=> FH + CHF2
# CF3-2 + CH2F2 <=> CHF3-2 + CHF2
# CHF2-2 + CH2O <=> CH2F2-2 + CHO
# HO + CH3F <=> H2O + CH2F
# HO + CHFO <=> H2O + CFO
# CF3-2 + H2O2 <=> CHF3-2 + HO2
# CH2F-2 + H2O2 <=> CH3F-2 + HO2
# H + CH2F2 <=> H2 + CHF2
# F + CHF3 <=> FH + CF3
# CF3-2 + HO-4 <=> CHF3-2 + O-2
# CFO-2 + CH3F <=> CHFO-2 + CH2F
# CF3-2 + CH4 <=> CHF3-2 + CH3
# CHF2-2 + H2O2 <=> CH2F2-2 + HO2
# CH2F-2 + CH2O <=> CH3F-2 + CHO
# CHF2-2 + CH3F <=> CH2F2-2 + CH2F
# CF3-2 + CH2O <=> CHF3-2 + CHO
# HO + CH2F2 <=> H2O + CHF2
# CF3-2 + CHFO <=> CHF3-2 + CFO
# H + CH3F <=> H2 + CH2F
# CFO-2 + H2O2 <=> CHFO-2 + HO2
# CH3-2 + CH2F2 <=> CH4-2 + CHF2
# CF3-2 + CH3F <=> CHF3-2 + CH2F
# CH3-2 + CHFO <=> CH4-2 + CFO
# CH3-2 + CH3F <=> CH4-2 + CH2F
# CFO-2 + CH2F2 <=> CHFO-2 + CHF2
# F + CHFO <=> FH + CFO
# HO + CHF3 <=> H2O + CF3
# CF3-2 + H2-2 <=> CHF3-2 + H-2
# O + CHFO <=> HO-2 + CFO
# H + CHFO <=> H2 + CFO